In [2]:
import numpy as np
import pandas as pd

In [5]:
annotated_training_data_path = '../../../Dataset/Annotated_Dataset/train_annotated.csv'
annotated_training_data_label_corrected_path = '../../../Dataset/Annotated_Dataset/train_annotated_label_corrected.csv'

In [7]:
annotated_training_data = pd.read_csv(annotated_training_data_path)
annotated_training_data.head()

,file_id,toc_section,toc_section_pos,toc_section_len,is_section_in_summary
0,10023,Highlights,37,60,True
1,10023,The Company at a glance,97,88,True
2,10023,Our strategy for growth,185,37,False
3,10023,Chairman’s statement,222,160,True
4,10023,Chief Executive’s report,382,187,True


In [8]:
annotated_training_data.is_section_in_summary.value_counts()

False    49774
True     18119
Name: is_section_in_summary, dtype: int64

In [10]:
toc_section_values = annotated_training_data['toc_section'].unique()
toc_section_values

array(['Highlights', 'The Company at a glance', 'Our strategy for growth',
       ..., 'Index to the consolidated and',
       'REPORT ON CORPORATE GOVERNANCE',
       'CONSOLIDATED STATEMENT OF TOTAL'], dtype=object)

In [11]:
annotated_training_data[annotated_training_data['toc_section'] == 'Highlights']['is_section_in_summary'].value_counts()

True     658
False    216
Name: is_section_in_summary, dtype: int64

In [12]:
def label_correction_using_majority_threshold(data_path, toc_section_name, label_col_name, corrected_data_path):
    
    df = pd.read_csv(data_path)
    toc_section_values = df[toc_section_name].unique()
    print('Number of sections to be processed: ', toc_section_values.shape)
    
    num = 0
    for toc_section_value in toc_section_values:
        value_counts = df[df[toc_section_name] == toc_section_value][label_col_name].value_counts()
        try:
            total_counts = value_counts.at[False] + value_counts.at[True]
            majority_value = True
            if value_counts.at[False] > value_counts.at[True]:
                majority_value = False
            
            per_majority = value_counts.at[majority_value]/total_counts
            
            if per_majority >= 0.7:
                df.loc[df[toc_section_name] == toc_section_value, label_col_name] = majority_value
            
        except Exception as e:
            #print(e)
            pass
        
        if num%500 == 0:
            print('Processed: ',toc_section_value, ' ',  num+1)
        num = num + 1
    
    
    df.to_csv(corrected_data_path, index=False)

In [13]:
label_correction_using_majority_threshold(annotated_training_data_path, 'toc_section', 'is_section_in_summary', annotated_training_data_label_corrected_path)

Number of sections to be processed:  (12958,)
Processed:  Highlights   1
Processed:  Corporate Governance Report   501
Processed:  Contents to financial statements   1001
Processed:  Phoenix Platinum   1501
Processed:  Notes to the Financial Information   2001
Processed:  Projects and pipeline   2501
Processed:  Business relationships   3001
Processed:  Report of the Independent Auditor to the Members of Sports Direct International plc   3501
Processed:  Area of operation   4001
Processed:  Ultra Electronics at a Glance   4501
Processed:  Directors’ responsibility for   5001
Processed:  Global market trends   5501
Processed:  EnQuest’s business model   6001
Processed:  BUSINESS MODEL   6501
Processed:  – Annual report on remuneration   7001
Processed:  SME Commercial Mortgages   7501
Processed:  Particulars of subsidiaries and associates   8001
Processed:  Ofﬁ ces and contacts   8501
Processed:  Trading market   9001
Processed:  Main board   9501
Processed:  Risks and their management 

In [14]:
annotated_training_data_label_corrected = pd.read_csv(annotated_training_data_label_corrected_path)

In [15]:
annotated_training_data_label_corrected[annotated_training_data_label_corrected['toc_section'] == 'Highlights']['is_section_in_summary'].value_counts()

True    874
Name: is_section_in_summary, dtype: int64

In [16]:
annotated_training_data_label_corrected[annotated_training_data_label_corrected['toc_section'] == 'Chairman’s statement']['is_section_in_summary'].value_counts()

True    1132
Name: is_section_in_summary, dtype: int64

In [17]:
annotated_training_data_label_corrected.is_section_in_summary.value_counts()

False    54206
True     13687
Name: is_section_in_summary, dtype: int64